# Work with textual data: encyclicals example

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import os
from collections import defaultdict

In [3]:
folder = '/Users/flint/Data/vatican/data'
data_collection = {}
for subdir in os.listdir(folder):
    pope = subdir
    data_collection[pope] = {}
    path = os.path.join(folder, pope, 'encyclicals')
    for doc in os.listdir(path):
        subpath = os.path.join(path, doc, 'it', '{}.txt'.format(doc))
        with open(subpath, 'r') as infile:
            content = infile.read()
        data_collection[pope][doc] = content

In [ ]:
print(list(data_collection.keys()))

In [ ]:
print(list(data_collection['Paul VI'].keys()))

In [ ]:
print(data_collection['Paul VI']['Humanae Vitae'][:1000])

## Tokenization and frequency

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
import spacy
nlp = spacy.load('it_core_news_lg')

In [ ]:
document = data_collection['Paul VI']['Humanae Vitae']

### Space tokenizer

In [ ]:
"questa è un'albicocca".split(), "questa è albicocca".split()

### Regex tokenizer

In [ ]:
word_tokenize("testo in cui si parla di albicocca.", language='italian')

In [ ]:
word_tokenize("testo in cui si parla di albicocche.", language='italian')

### Syntactic tokenizer

In [ ]:
from spacy.displacy import render

In [ ]:
doc = nlp(document)

In [ ]:
sentences = [sentence for sentence in doc.sents]

In [ ]:
print(sentences[21])

In [ ]:
for token in sentences[21]:
    print(token, token.lemma_, token.pos_)

In [ ]:
print(sentences[21])
render(sentences[21])

## Processo di tokenizzazione
- Filtro: togliamo punteggiatura PUNCT, articoli DET, ADP
- Con lemmatizzazione

In [ ]:
def tokenize(text, lemmatize=True, filter_pos=None):
    bow = defaultdict(lambda: 0)
    doc = nlp(text)
    for sentence in doc.sents:
        for token in sentence:
            if lemmatize:
                word = token.lemma_
            else:
                word = token.text
            if filter_pos is None:
                bow[word] += 1
            else:
                if token.pos_ not in filter_pos:
                    bow[word] += 1
    return pd.Series(bow)

In [ ]:
bow = tokenize(data_collection['Paul VI']['Humanae Vitae'], 
               filter_pos=['PUNCT', 'DET', 'ADP'])

In [ ]:
bow.sort_values(ascending=False).head(20)

In [ ]:
corpus_index = {}
popes = ['John XXIII', 'Paul VI']
for pope, corpus in data_collection.items():
    if pope in popes:
        corpus_index[pope] = {}
        for document, content in corpus.items():
            freq = tokenize(content)
            corpus_index[pope][document] = freq

In [ ]:
hum_vit = corpus_index['Paul VI']['Humanae Vitae']
pax = corpus_index['John XXIII']['Pacem']

In [ ]:
hum_vit.sort_values(ascending=False)['uomo'] / hum_vit.sum()

In [ ]:
hum_vit.sort_values(ascending=False)['che'] / hum_vit.sum()

In [ ]:
pax.sort_values(ascending=False)['uomo'] / pax.sum()

In [ ]:
pax.sort_values(ascending=False)['che'] / pax.sum()

In [ ]:
hum_vit_norm = hum_vit / hum_vit.sum()

In [ ]:
hum_vit_norm.sort_values(ascending=False)

## Document frequency

In [ ]:
docu_freq = defaultdict(lambda: 0)
for pope, corpus in corpus_index.items():
    for document, idx in corpus.items():
        for word in idx.keys():
            docu_freq[word] += 1
DF = pd.Series(docu_freq)

In [ ]:
DF.sort_values(ascending=False).head(20)

In [ ]:
(15 / DF).sort_values(ascending=False)